#ASOS 종관 정보 다운

In [1]:
import requests
from urllib import parse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as d
import numpy as np

In [2]:
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = 'SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D'

In [3]:
params = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
    parse.quote_plus('pageNo') : '1', 
    parse.quote_plus('numOfRows') : '10', 
    parse.quote_plus('dataType') : 'XML', 
    parse.quote_plus('dataCd') : 'ASOS', 
    parse.quote_plus('dateCd') : 'HR', 
    parse.quote_plus('startDt') : '20211225', 
    parse.quote_plus('startHh') : '00', 
    parse.quote_plus('endDt') : '20211226', 
    parse.quote_plus('endHh') : '01', 
    parse.quote_plus('stnIds') : '104' 
    })

In [4]:
response = requests.get(url + params).text
#response = requests.get(url + params)
#.text.encode('utf-8')
print(url + params)

http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=HR&startDt=20211225&startHh=00&endDt=20211226&endHh=01&stnIds=104


In [5]:
soup = BeautifulSoup(response, 'html.parser')
tm = soup.select('body > items > item > tm')
ta = soup.select('body > items > item > ta')
dsnw = soup.select('body > items > item > dsnw')

In [6]:
dsnw = soup.select('body > items > item')
dsnw

[<item><tm>2021-12-25 00:00</tm><rnum>1</rnum><stnid>104</stnid><stnnm>북강릉</stnnm><ta>0.4</ta><taqcflg></taqcflg><rn>11.6</rn><rnqcflg></rnqcflg><ws>1.5</ws><wsqcflg></wsqcflg><wd>320</wd><wdqcflg></wdqcflg><hm>99</hm><hmqcflg></hmqcflg><pv>6.2</pv><td>0.2</td><pa>1011.2</pa><paqcflg></paqcflg><ps>1020.7</ps><psqcflg></psqcflg><ss></ss><ssqcflg>9</ssqcflg><icsr></icsr><dsnw>12.9</dsnw><hr3fhsc>11.9</hr3fhsc><dc10tca>10</dc10tca><dc10lmcsca>10</dc10lmcsca><clfmabbrcd>St</clfmabbrcd><lcsch>1</lcsch><vs>120</vs><gndsttcd></gndsttcd><dmstmtphno>05</dmstmtphno><ts>0.4</ts><tsqcflg></tsqcflg><m005te>1.6</m005te><m01te>2.6</m01te><m02te>4.0</m02te><m03te>4.6</m03te></item>,
 <item><tm>2021-12-25 01:00</tm><rnum>2</rnum><stnid>104</stnid><stnnm>북강릉</stnnm><ta>0.2</ta><taqcflg></taqcflg><rn></rn><rnqcflg>9</rnqcflg><ws></ws><wsqcflg>9</wsqcflg><wd></wd><wdqcflg>9</wdqcflg><hm>98</hm><hmqcflg></hmqcflg><pv>6.1</pv><td>0.0</td><pa>1011.0</pa><paqcflg></paqcflg><ps>1020.5</ps><psqcflg></psqcflg><s

In [9]:
rows = []
for idx in range(len(tm)):
    rows.append({"tm": tm[idx], 
                 "ta": ta[idx], 
                 "dsnw": dsnw[idx]
                 })

In [10]:
columns = ["tm", "ta", "dsnw"]
catalog_cd_df = pd.DataFrame(rows, columns = columns)
catalog_cd_df.head(10)

,tm,ta,dsnw
0,[2021-12-25 00:00],[0.4],"[[2021-12-25 00:00], [1], [104], [북강릉], [0.4],..."
1,[2021-12-25 01:00],[0.2],"[[2021-12-25 01:00], [2], [104], [북강릉], [0.2],..."
2,[2021-12-25 02:00],[-0.1],"[[2021-12-25 02:00], [3], [104], [북강릉], [-0.1]..."
3,[2021-12-25 03:00],[-0.4],"[[2021-12-25 03:00], [4], [104], [북강릉], [-0.4]..."
4,[2021-12-25 04:00],[-0.5],"[[2021-12-25 04:00], [5], [104], [북강릉], [-0.5]..."
5,[2021-12-25 05:00],[-0.8],"[[2021-12-25 05:00], [6], [104], [북강릉], [-0.8]..."
6,[2021-12-25 06:00],[-1.0],"[[2021-12-25 06:00], [7], [104], [북강릉], [-1.0]..."
7,[2021-12-25 07:00],[-2.4],"[[2021-12-25 07:00], [8], [104], [북강릉], [-2.4]..."
8,[2021-12-25 08:00],[-2.6],"[[2021-12-25 08:00], [9], [104], [북강릉], [-2.6]..."
9,[2021-12-25 09:00],[-3.8],"[[2021-12-25 09:00], [10], [104], [북강릉], [-3.8..."


# 날짜별 다운로드 함수

In [ ]:
class makingParam:
  def set_my_params(key, date, place):
    params = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
        parse.quote_plus('pageNo') : '1', 
        parse.quote_plus('numOfRows') : '10', 
        parse.quote_plus('dataType') : 'XML', 
        parse.quote_plus('dataCd') : 'ASOS', 
        parse.quote_plus('dateCd') : 'HR', 
        parse.quote_plus('startDt') : date, 
        parse.quote_plus('startHh') : '00', 
        parse.quote_plus('endDt') : date+1, 
        parse.quote_plus('endHh') : '01', 
        parse.quote_plus('stnIds') : place 
    })
  return params

def download_file(year, month):  
    # initialize the date
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    key = 'SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D'
    param = makingParam()
    if month == 12:
        day_count = 31
    else:
        day_count = (d.date(year, month+1, 1) - d.date(year, month, 1)).days
    interval = d.timedelta(minutes=60)
    place = 103

    for i in range(0, day_count):
        im_tensor = np.zeros((144, 10))  
        day = i+1
        
        for j in range(0, 144):
            start_date = d.datetime(year, month, day, 0, 0)
            date_time = (start_date + j * interval)
            datetime = date_time.strftime("%Y%m%d%H")
            my_param = param(key, datetime, place)
            response = requests.get(url + my_param).text
            soup = BeautifulSoup(response, 'html.parser')
            items = soup.find_all('body > item')

            for k in range(len(items)):
              tm = soup.select('body > items > item > tm')
        
download_file(2022, 1) 